#### [교차검증 (Cross Validation) - cross_val_score() / cross_validate()]
- 적은 데이터셋으로 안정적이고 신뢰성 있는 모델 평가를 위한 방법
- 학습 데이터셋을 K개 분할 후 매번 다른 데이터로 검증 진행
- 교차검증 후 모델의 일반화 성능으로 여김
- sklearn.modl_selection 서브모듈에 존재하는 함수들
    * cross_val_score() / cross_val_predict() : cv만큼에 대한 성능결과, 예측결과 반환
    * cross_validate() : 다양한 정보들 반환, 많이 사용됨!

[1] 모듈 로딩 및 데이터 준비 <hr>

In [156]:
# ----------------------------------------------------
# [1-1] 모듈 로딩
# ----------------------------------------------------
# 분석
import pandas as pd
import numpy as np

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 머신러닝 
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate  # 교차검증
from sklearn.neighbors import KNeighborsClassifier                                      # 학습 알고리즘(KNN)

# ----------------------------------------------------
# [1-2] 데이터 준비
# ----------------------------------------------------
FILE_NAME = '../Data/iris.csv'
irisDF = pd.read_csv(FILE_NAME)
display(irisDF.head(3))

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


[2] 데이터 전처리 및 학습 준비: 시간 때문에 미진행 => 개인별로 진행

In [157]:
# ----------------------------------------------------
# [2-1] 품종컬럼 자료형 변환
# ----------------------------------------------------
pd.options.mode.copy_on_write = True
irisDF.variety = irisDF.variety.astype('category')
print(irisDF.info())

# ----------------------------------------------------
# [2-2] 피쳐와 타겟 분리
# ----------------------------------------------------
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]

print(f"featureDF : {featureDF.shape}, targetSR : {targetSR.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   sepal.length  150 non-null    float64 
 1   sepal.width   150 non-null    float64 
 2   petal.length  150 non-null    float64 
 3   petal.width   150 non-null    float64 
 4   variety       150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB
None
featureDF : (150, 4), targetSR : (150,)


In [158]:
# ---------------------------------------------------------------------
# [2-3] 수치형 컬럼 => 학습 알고리즘에 따라 적용
#                 => KNN 알고리즘은 거리 측정 기반으로 스케일링이 필요함
# ---------------------------------------------------------------------

[3] 교차검증 <HR>

In [ ]:
# ------------------------------------------------------------------
# [3-1] 
# cross_validate() 함수 : 전달된 데이터에 따라서 자동으로 KFold, StratifiedKFold 설정
#
# - 필수 매개변수
#   estimator          : 모델 인스턴스
#   cv                 : 5(기본값) or KFold, StratifiedKFold 인스턴스 -> 교차 검증 폴드 수
#   return_train_score : 학습용 데이터셋 성능 반환여부 설정
#   return_estimator   : 학습 모델 인스턴스
# ------------------------------------------------------------------
# 모델 인스턴스 생성
MAX_N = 21

resultDF = pd.DataFrame(columns=['fit_time', 'score_time', 'test_score', 'train_score', 'neighbors', 'diff'])

for n in range(1, MAX_N):
    # 1. 모델 생성
    k_model = KNeighborsClassifier(n_neighbors=n)

    # 2. 교차검증 수행
    result_dict = cross_validate(
        k_model,
        featureDF,
        targetSR,
        return_train_score=True,
        cv=3
    )

    # 3. 평균값 계산
    fit_time_mean   = result_dict['fit_time'].mean()
    score_time_mean = result_dict['score_time'].mean()
    test_mean       = result_dict['test_score'].mean()
    train_mean      = result_dict['train_score'].mean()

    # 4. diff 계산
    diff = abs(train_mean - test_mean)

    # 5) 한 행(row)을 리스트로 만들어 DataFrame에 추가
    row = [fit_time_mean, score_time_mean, test_mean, train_mean, n, diff]
    resultDF.loc[len(resultDF)] = row

In [160]:
display(resultDF.head())

,fit_time,score_time,test_score,train_score,neighbors,diff
0,0.004256,0.001018,0.960000,1.000000,1.0,0.040000
1,0.000770,0.000000,0.946667,0.976667,2.0,0.030000
2,0.000609,0.001370,0.973333,0.963333,3.0,0.010000
3,0.000668,0.000982,0.986667,0.960000,4.0,0.026667
4,0.004833,0.002026,0.980000,0.970000,5.0,0.010000
